In [4]:
from CNN_model import model
from keras.models import Model
import numpy as np
import matplotlib.image as mpimg
from consts import *
import glob
from sklearn import svm

model.load_weights(weights_dir + 'cnn_' + str(CLASS_ID) + '.h5')
train_path = base_path + 'images/cropped_resized/train/'
test_path = base_path + 'images/cropped_resized/test/'
svm_weights_path = base_path + 'weights/svm_' + str(CLASS_ID) + '.joblib'

train_vecs = []
train_labels = []
n = 0
nmax = 100000

intermed_model = Model(inputs=model.input,
                   outputs=model.layers[-2].output)

for filepath in glob.glob(train_path + '*.*'):
    if n > nmax:
        break
    im = mpimg.imread(filepath)
    im -= 0.5
    im = np.expand_dims(im, axis=0)
    layer_output = intermed_model(im)
    train_vecs.append(layer_output[0])

    if CLASS_NAMES[CLASS_ID] in filepath:
        train_labels.append(1)
    else:
        train_labels.append(0)

    n += 1


train_vecs = np.array(train_vecs)
print(train_vecs.shape)


(14783, 512)


In [5]:
clf = svm.SVC(kernel='linear', probability=True)
clf.fit(train_vecs, train_labels)

SVC(kernel='linear', probability=True)

In [6]:
from joblib import dump, load
dump(clf, svm_weights_path) 

['./cuttlefish_data/weights/svm_0.joblib']

In [7]:
test_vecs = []
test_labels = []
n = 0
nmax = 100000
for filepath in glob.glob(test_path + '*.*'):
    if n > nmax:
        break
    im = mpimg.imread(filepath)
    im -= 0.5
    im = np.expand_dims(im, axis=0)
    layer_output = intermed_model(im)
    test_vecs.append(layer_output[0])

    if CLASS_NAMES[CLASS_ID] in filepath:
        test_labels.append(1)
    else:
        test_labels.append(0)
    n += 1

svm_preds = clf.predict(test_vecs)
svm_probs = clf.predict_proba(test_vecs)

# print(test_res)
# print(test_labels)
wrong_count = 0
for svm_prob, true_label in zip(svm_probs, test_labels):
    if svm_prob[1] > 0.7:
        svm_guess = 1
    else:
        svm_guess = 0
    if svm_guess != true_label:
        wrong_count += 1
print(str(wrong_count) + '/' + str(len(test_labels)))


49/1851
